**Lab III - Advanced Topics**

Workshop III

1. *In your own words, describe what vector embeddings are and what they are useful for*

Los embeddings son como mapas que convierten palabras, oraciones e incluso imágenes en puntos en un espacio 3D. Estos puntos se agrupan según su significado, permitiendonos usar la logica matemáticas para compararlos y analizarlos. Son muy útiles para búsquedas en internet, traducción automática, generación de imágenes a partir de texto y control de robots. En resumen, los embeddings son una herramienta poderosa para que las máquinas comprendan el mundo que nos rodea.

Dicha de otra manera Imagina que tienes una biblioteca gigante con miles de libros. Los embeddings te ayudan a encontrar el libro que buscas sin tener que leer cada página. Simplemente describes el libro que necesitas, y el embedding te guía hacia la sección de la biblioteca donde se encuentra. Pareciera que tiene un bibliotecario experto a su disposición.

Los embeddings están cambiando la forma en que las máquinas interactúan con el mundo. Son una herramienta poderosa que nos permite aprovechar el poder de la inteligencia artificial para resolver problemas complejos y mejorar nuestras vidas.

2. *What do you think is the best distance criterion to estimate how far two embeddings (vectors) are from each  other? Why?*

A la hora de seleccionar el criterio óptimo para medir la distancia entre nuestros vectores de incrustación, se plantea una dicotomía clásica entre la distancia euclidiana y la distancia del coseno. Esta elección se asemeja a la decisión entre calcular la distancia en línea recta, una opción más directa y concisa (distancia euclidiana), o considerar el ángulo entre los vectores, valorando la similitud en la dirección de sus movimientos (distancia del coseno). Podríamos ilustrar esta disyuntiva imaginando la eficiencia de un trayecto directo entre estantes de libros en una biblioteca frente a una ruta más sutil y circular. En el ámbito de las incrustaciones de texto, donde el interés radica en capturar la esencia de la similitud semántica, la preferencia por la distancia del coseno es evidente. Esta métrica atiende al estilo del baile más que a los movimientos individuales, convirtiéndola en una opción clave cuando la atención recae en la esencia de la similitud, especialmente en el dinámico y dimensionalmente elevado terreno del procesamiento del lenguaje natural. Así, la elección entre una línea recta o un arco elegante depende del ritmo inherente a los datos y de la coreografía específica de la tarea en cuestión

3. Let us build a Q&A (question answering) system! 😀For this, consider the following steps:
a. Pick whatever text you like, in the order of 20+ paragraphs
b. Split that text into meaningful chunks/pieces
c. Implement the embedding generation logic. Which tools and approaches would help you generate them easily and high-level?
d. For every question asked by the user, return a sorted list of the N chunks/pieces in your text that relate the most to the question. Do results make sense?
4. What do you think that could make these types of systems more robust in terms of semantics and functionality?
5. Bonus points if deployed on a local or cloud server.

Script: 

In [ ]:
import json
import torch #Libreria para operaciones de aprendizaje profundo
from transformers import AutoTokenizer, AutoModelForQuestionAnswering #tokenizador asociado al modelo y modelo pre-entrenado para tareas de respuesta a preguntas
from datasets import Dataset
import torch.nn.functional as F #cálculo de la similitud del coseno

In [1]:
class QAModel:
    def __init__(self, model_name):
        # Inicializa el modelo de pregunta y respuesta
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)# este es el tokenizador y carga el modelo 
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # determina si se utilizará la GPU o la CPU. El modelo se mueve a la GPU si está disponible.
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    # Toma un texto como entrada, tokeniza el texto utilizando el tokenizador, y obtiene los embeddings utilizando el modelo de pregunta y respuesta
    def generate_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)#Mueve la entrada tokenizada al dispositivo elegido
        inputs.to(self.device)
        with torch.no_grad(): # Desactiva el cálculo del gradiente para mayor eficiencia
            outputs = self.model(**inputs) #outputs = self.model(inputs)
            # Corregir a 'pooler_output' en lugar de 'last_hidden_state'
            # Calcula el embeding del párrafo utilizando el agrupamiento medio sobre el último estado oculto
            embeddings = outputs.pooler_output.squeeze() # embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
        return embeddings.to(self.device)
    # Utiliza la similitud del coseno para comparar los embeddings del input del usuario con los embeddings de cada párrafo.
    def find_best_match(self, user_input, paragraphs, k=3):
        user_embedding = self.generate_embedding(user_input)#Genera un embedings para la entrada del usuario utilizando
        best_matches = [] # Inicializa una lista vacía
        for i, paragraph in enumerate(paragraphs): # Itera sobre cada párrafo
            paragraph_embedding = self.generate_embedding(paragraph)
            # Calcula la similitud del coseno entre las incrustaciones del usuario y del párrafo usando (F.cosine_similarity)
            similarity = F.cosine_similarity(user_embedding.unsqueeze(0), paragraph_embedding.unsqueeze(0), dim=1).item()
            best_matches.append((i, similarity))#Añade el índice del párrafo y la similitud
        #best_matches.sort(key=lambda x: x[1], reverse=True) #Ordena la lista
        best_matches.sort(key=lambda x: x[1], reverse=True)
        return best_matches[:k] # Devuelve las k mejores coincidencias

In [ ]:
if __name__ == "__main__":
    # Cargar el texto desde un archivo
    file_path = 'C:\\Users\\Cesars\\OneDrive\\Escritorio\\Lab3\\Parrafo.txt'
    with open(file_path, 'r', encoding='utf-8') as file:
        texto = file.read() # Se almacena en la variable texto
    all_answers = []
    # Dividir el texto en párrafos significativos
    paragraphs = [p.strip() for p in texto.split("\n") if p.strip()] # Se divide en párrafos y se almacena en la lista

    qa_model = QAModel("deepset/roberta-base-squad2") # Crear instancia del modelo
    #carga, lee y almacena las preguntas en questions
    questions_file_path = 'C:\\Users\\Cesars\\OneDrive\\Escritorio\\Lab3\\Preguntas.json'
    with open(questions_file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        questions = data["preguntas"]

In [ ]:
for question in questions:
        # Para cada pregunta, se llama a la función find_best_match del modelo para encontrar las k mejores coincidencias con los párrafos en la lista paragraphs
        best_matches = qa_model.find_best_match(question, paragraphs)
        best_paragraph_index = best_matches[0][0]
        best_answer = paragraphs[best_paragraph_index]
        all_answers.append({"Pregunta": question, "Respuesta": best_answer})

In [ ]:
#Almacena cada Respuesta en un archivo json
respuesta_file_path = 'C:\\Users\\Cesars\\OneDrive\\Escritorio\\Lab3\\Respuesta.json'
with open(respuesta_file_path, "w", encoding="utf-8") as f:
    json.dump(all_answers, f, ensure_ascii=False, indent=2)